In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
file_path = './Dataset/OLA_DataSet.xlsx'

df = pd.read_excel(file_path)

In [ ]:
df.isna().sum()

In [ ]:
# changing dtype

Acc_type = {'Date': 'datetime64[ns]', 'Time': 'datetime64[ns]', 'Booking_ID': 'string', 'Booking_Status': 'category', 'Customer_ID' : 'string',
       'Vehicle_Type': 'Category', 'Pickup_Location': 'string', 'Drop_Location': 'string', 'V_TAT': 'int', 'C_TAT': 'int',
       'Canceled_Rides_by_Customer': 'string', 'Canceled_Rides_by_Driver' : 'strnig',
       'Incomplete_Rides': 'category', 'Incomplete_Rides_Reason': 'category', 'Booking_Value': 'int32',
       'Payment_Method': 'category', 'Ride_Distance': 'int32', 'Driver_Ratings': 'category', 'Customer_Rating': 'category'}

df = df.convert_dtypes(Acc_type)
df.info()

In [ ]:
# checking for duplicates 

df.duplicated().sum()

In [ ]:
# handling null values for ride status column
 
df['ride_status'] = df.apply(
    lambda x: 'incomplete' if (pd.isna(x['Incomplete_Rides'])) | (x['Incomplete_Rides'] == 'Yes')
              else 'complete', 
    axis=1
)
df['ride_status'].value_counts()

# Validation

In [ ]:
numeric_cols = df.select_dtypes(include='number')
for col in numeric_cols:
    print(col, df[col].min(), df[col].max())

# Transformation

In [ ]:
# 1. remove time from date columns
df['Date'] = pd.to_datetime(df['Date']).dt.date


In [ ]:
# 2. find actual reason for cancellation
reason_cols = ['Canceled_Rides_by_Driver', 'Canceled_Rides_by_Customer', 'Incomplete_Rides_Reason']
temp_reason = df[reason_cols].max(axis=1)

df['cancellation_reason'] = temp_reason.fillna(df['Booking_Status'])
df['cancellation_reason'] = df['cancellation_reason'].apply(lambda x: None if x == 'Success' else x)
df

In [ ]:
"""
3. change ride Distance to categories
    1-10 short distance
    11-25 Medium distance
    26-50 long distance
    """
    
df['distance_cat'] = df['Ride_Distance'].apply(lambda x: 'short' if x <= 10 else 'medium' if x <= 25 else 'long')
df.head()

# Outlier's and Distribution

In [ ]:
df.select_dtypes(include='number').plot(
    kind='box', 
    subplots=True, 
    layout=(2, 3), 
    figsize=(12, 8)
)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution
cols_to_plot = ['Vehicle_Type', 'Payment_Method', 'distance_cat', 'Driver_Ratings', 'Customer_Rating', 'cancellation_reason']

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

axes_flat = axes.flatten()

for i, col in enumerate(cols_to_plot):
    counts = df[col].value_counts()
    ax = counts.plot(kind='bar', ax=axes_flat[i], color='skyblue')
    
    ax.bar_label(ax.containers[0], padding=3)
    
    ax.set_title(f"Distribution: {col}")
    ax.set_ylabel("Count")
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
df.drop(columns=['Booking_ID', 'Vehicle Images', 'Canceled_Rides_by_Customer', 'Canceled_Rides_by_Driver', 'Booking_Status', 'Incomplete_Rides_Reason', 'Incomplete_Rides'], inplace=True)

# DataCleaning & Preprocessing
●Handle missing/inconsistent values. 

●Convert datatypes and standardize formats. 

●Create derived features if necessary for better insights. 



In [ ]:
# save df to csv

df.to_csv('./Dataset/OLA_clean.csv', index=False)

# SQL Query Development

●Write queries to extract insights(e.g.,ridetrends, cancellations, ratings).

●Optimize queries for performance and accuracy.

●Validate results against the dataset.


In [ ]:
df.columns

In [ ]:
# connect this file to server

!pip install pandas sqlalchemy pyodbc mssql-python -q

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Float, Integer, String, DateTime, text
import mssql_python



connection_str = (
    "Server=PARADOX;"
    "Database=OLA;"
    "Authentication=ActiveDirectoryIntegrated;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)

engine = create_engine(
    "mssql+pyodbc://", 
    creator=lambda: mssql_python.connect(connection_str),
    fast_executemany=True
)
with engine.connect() as conn:
    # Use IF EXISTS to avoid errors if the table is already gone
    conn.execute(text("IF OBJECT_ID('dbo.Rides', 'U') IS NOT NULL DROP TABLE dbo.Rides"))
    conn.commit() 
# 3. Send the DataFrame
try:
    df.to_sql(
        name='Rides', 
        con=engine, 
        schema='dbo',
        if_exists='replace', 
        index=False, 
        chunksize=1000
    )
    print("Upload successful!")
except Exception as e:
    print(f"Connection failed: {e}")
